# General Information about the Excel sheet and Sequemce files
#### The dataset is not homogenous, in other words, it has different naming system and different formats among Excel and seq. files, so this jupyter will focus on standadrize the dataset

### 1. The SEequence files:
#### 1. File Format:
The files format vary among the folders to:
- fasta
- txt
- gb
#### 2. File names as following:
Each folder is named after specific genotype (B3), and in each folder, the file names are:
*yy-nnnnn.xx A*
- yy- : year e.g. 00- = 20(00)
- -nnnnn- : is the sequential number of package
- .xx is sequential number of the sample from that package (sampling type)
- A : String stand to the type

The *yy-nnnnn* is fixed among the files' name.

### 2. The Excel sheet:
| SCount| Ein | EingangsDatum | EntnahmeDatum | ErkrBeginn | EkzBeginn | BLand | GesAmt | Material | Genotyp | WHO distinct SeqID |
|----------|----------|----------|----------|----------|----------|----------|----------|----------|----------|----------|
| 06-00025.01  | 70794 Filderstadt, Gemeinschaftspraxis  | 2/3/2006  | 2/1/2006  | 1/28/2006 | 1/31/2006 |Baden-Württemberg	| NAN | oral fluid | B3 | 2254 |

- SCount: patient ID (*yy-nnnnn.xx*) as in the file name
- Ein: Location of Sampling
- EingangsDatum: Date of entry
- EntnahmeDatum: Date of withdrawal 
- ErkrBeginn: Start of illness 
- EkzBeginn: Start of illness 
- BLand: Federal state  
- Total : ?

*yy-nnnnn.xx* is fixed along the excel sheet.

### 3. The issue is:
In general the naming of the files are not consistent across the folder, and doesn't match the excel sheet. As excel sheet have only yy-nnnnn.xx the serial number nothing added, and only 1 patient sample. While the folder has reptitive sequence for different samples and the name is not standardized across the file.



## 1. Changing file format to FASTA formats
Check the files (FASTA format and naming)
### 1. Packages:
- Biopython: to make convert files to FASTA (https://biopython.org/wiki/SeqIO)  
- Fasta Validator: to make sure that the conversion is successful
```Python
        conda install -c bioconda py_fasta_validator
```
or
```Python
        pip install py-fasta-validator
```
### 2. Changing formats

### 3. Validator: 
The following return codes:

```0``` this is a valid fasta file

```1``` the first line does not start with a > (rule 1 violated).

```2``` there are duplicate sequence identifiers in the file (rule 7 violated)

```4``` there are characters in a sequence line other than [A-Za-z]

#### 1. Importation of packages:

In [7]:
#!pip install py-fasta-validator

In [8]:
import pandas as pd
from Bio import SeqIO
import glob
import os
import FastaValidator
import subprocess
import csv
from collections import Counter
from marshmallow import fields, ValidationError
import matplotlib.pyplot as plt

#### 2. Change Formates to FASTA

1. Output directory path

In [9]:
#output_folder = input("Output directory:")
output_folder = "/Users/MiladM-Dev/Documents/1PhD/Dataset_trials/B3_mimic/results"

2. Convert *gb* to fasta

In [10]:
# Input and output paths
#input_folder = input("genebank directory:")
input_folder = "/Users/MiladM-Dev/Documents/1PhD/Dataset_trials/B3_mimic/gene_data/"

# Match all GenBank files in the input folder
genbank_files = glob.glob(os.path.join(input_folder, "*.gb"))

# Convert each GenBank file to FASTA
for gb_file in genbank_files:
    base_name = os.path.splitext(os.path.basename(gb_file))[0]  # Get the base file name
    fasta_file = os.path.join(output_folder, f"{base_name}.fasta")
    
    SeqIO.convert(gb_file, "genbank", fasta_file, "fasta")
    print(f"Converted {gb_file} to {fasta_file}")



Converted /Users/MiladM-Dev/Documents/1PhD/Dataset_trials/B3_mimic/gene_data/13-00122Rsp 1.gb to /Users/MiladM-Dev/Documents/1PhD/Dataset_trials/B3_mimic/results/13-00122Rsp 1.fasta
Converted /Users/MiladM-Dev/Documents/1PhD/Dataset_trials/B3_mimic/gene_data/13-00122Rsp copy 1.gb to /Users/MiladM-Dev/Documents/1PhD/Dataset_trials/B3_mimic/results/13-00122Rsp copy 1.fasta


3. Convert *txt* to fasta

In [11]:
#input_folder = input("genebank directory:")
input_folder = "/Users/MiladM-Dev/Documents/1PhD/Dataset_trials/D8_mimic/txt"


def txt_to_fasta(input_file, output_file):

    with open(input_file, 'r') as txt_file, open(output_file, 'w') as fasta_file:
        for line in txt_file:
            line = line.strip()
            if not line:
                continue  # Skip empty lines
            if line.startswith(">"):
                # If line starts with ">", it's a header
                fasta_file.write(f"{line}\n")
            else:
                # Otherwise, treat it as a sequence
                fasta_file.write(f"{line}\n")

# Example usage
text_files = glob.glob(os.path.join(input_folder, "*.txt"))

# Convert each txt file to FASTA
for text_file in text_files:
    base_name = os.path.splitext(os.path.basename(text_file))[0]  # Get the base file name
    fasta_file = os.path.join(output_folder, f"{base_name}.fasta")
    
    txt_to_fasta(text_file, fasta_file)
    print(f"Converted {text_file} to {fasta_file}")


### 3. FASTA validator

a. Validate FASTA files

to check conversion is successful

In [13]:
fasta_dir = output_folder
output_testfile = "/Users/MiladM-Dev/Documents/1PhD/Dataset_trials/B3_mimic/results/"
csv_output = "/Users/MiladM-Dev/Documents/1PhD/Dataset_trials/B3_mimic/results/validation_results.csv"

# Open the text and CSV output files
with open(output_testfile, "w") as out_file, open(csv_output, "w", newline="") as csv_file:
    # Create a CSV writer object
    csv_writer = csv.writer(csv_file)
    # Write the CSV header
    csv_writer.writerow(["File Name", "Exit Code"])

    # Iterate over all .fasta files in the directory
    for fasta_file in glob.glob(os.path.join(fasta_dir, "*.fasta")):
        out_file.write(f"{fasta_file}\n")
        print(f"{fasta_file}")

        # Run the py_fasta_validator command
        result = subprocess.run(
            ["py_fasta_validator", "-f", fasta_file],
            stdout=subprocess.PIPE,
            stderr=subprocess.PIPE,
            text=True
        )

        # Print and log the validation output (stdout and stderr)
        if result.stdout.strip():
            out_file.write(f"{result.stdout.strip()}\n")
            print(result.stdout.strip())
        if result.stderr.strip():
            out_file.write(f"{result.stderr.strip()}\n")
            print(result.stderr.strip())

        # Log and print the exit code
        out_file.write(f"Exit code: {result.returncode}\n\n")
        print(f"Exit code: {result.returncode}")

        # Write the file name and exit code to the CSV file
        csv_writer.writerow([os.path.basename(fasta_file), result.returncode])

print(f"CSV summary created at: {csv_output}")


IsADirectoryError: [Errno 21] Is a directory: '/Users/MiladM-Dev/Documents/1PhD/Dataset_trials/B3_mimic/results/'

C. Counting the sample names in the excel sheet

In [ ]:
# Folder containing the files
folder_dir = "/Users/MiladM-Dev/Documents/1PhD/Dataset_trials/D8_mimic/fasta/"

# Extract strings from filenames and count occurrences
def extract_and_count(folder_path):
    string_counts = Counter()
    
    for filename in os.listdir(folder_path):
        # Ensure we are working only with files
        if os.path.isfile(os.path.join(folder_path, filename)):
            # Split filename to extract the string part after numbers
            # Assuming the format is similar to "13-00125Rspcopy.fasta"
            base_name, _ = os.path.splitext(filename)  # Remove file extension
            parts = base_name.split("-", 1)  # Split at the first dash
            
            if len(parts) > 1:  # Ensure there is a string part after the dash
                string_part = ''.join(filter(str.isalpha, parts[1]))  # Keep only the string part
                string_counts[string_part] += 1

    return string_counts

# Write the counts to a CSV file
def write_to_csv(output_file, counts):
    with open(output_file, mode='w', newline='') as csv_file:
        writer = csv.writer(csv_file)
        writer.writerow(["String", "Count"])  # Write header
        
        for string, count in counts.items():
            writer.writerow([string, count])  # Write each string and its count

# Main execution
if __name__ == "__main__":
    counts = extract_and_count(folder_dir)
    output_csv = os.path.join(folder_dir, "string_counts.csv")
    write_to_csv(output_csv, counts)
    print(f"String counts saved to {output_csv}")


String counts saved to /Users/MiladM-Dev/Documents/1PhD/Dataset_trials/D8_mimic/fasta/string_counts.csv


# 2. Check the names in the files

# Write the pipline here

### 1. Validation of file names  (Marshamallow)

In [14]:
class FileNameField(fields.Str):
    def _validate(self, value):
        # Validate file extension
        if not value.endswith(".fasta"):
            raise ValidationError(f"Invalid file extension for '{value}'. Expected '.fasta'.")

        try:
            # Remove the file extension and split the base name
            base_name = value[:-6]  # Remove '.fasta'
            yy, rest = base_name.split('-')
            nnnnn, xx = rest.split('.')

            # Ensure components are of the correct format
            if not (yy.isdigit() and len(yy) == 2):
                raise ValidationError(f"Invalid 'yy' value: '{yy}'. Must be a two-digit number.")
            if not (nnnnn.isdigit() and len(nnnnn) == 5):
                raise ValidationError(f"Invalid 'nnnnn' value: '{nnnnn}'. Must be a five-digit number.")
            if not (xx.isdigit() and len(xx) == 2):
                raise ValidationError(f"Invalid 'xx' value: '{xx}'. Must be a two-digit number.")

            # If needed, you can process or store these components
            self.yy = yy
            self.nnnnn = nnnnn
            self.xx = xx

        except ValueError:
            raise ValidationError(f"'{value}' does not match the required format 'yy.nnnnn-xx.fasta'.")

def validate_files_in_folder(folder_path):
    field = FileNameField()
    valid_files = []
    invalid_files = []
    csv_output = os.path.join(folder_path, "validation_summary.csv")

    for file_name in os.listdir(folder_path):
        if os.path.isfile(os.path.join(folder_path, file_name)):  # Ensure it's a file
            try:
                field._validate(file_name)  # Validate the file name
                valid_files.append(file_name)
            except ValidationError as e:
                invalid_files.append((file_name, str(e)))

    # Print results
    #print("\nInvalid file names:")
    #for file, error in invalid_files:
    #   print(f"{file}: {error}")

    #print("\nValid file names:")
    #for file in valid_files:
    #   print(file)

    # Write results to CSV
    with open(csv_output, "w", newline="") as csv_file:
        csv_writer = csv.writer(csv_file)
        csv_writer.writerow(["File Name", "Status", "Error Message"])
        for file_name in valid_files:
            csv_writer.writerow([file_name, "Valid", ""])
        for file_name, error in invalid_files:
            csv_writer.writerow([file_name, "Invalid", error])

    print(f"\nCSV summary created at: {csv_output}")

if __name__ == "__main__":
    folder_path = '/Users/MiladM-Dev/Documents/1PhD/Dataset_trials/B3_mimic/fasta'
    validate_files_in_folder(folder_path)



CSV summary created at: /Users/MiladM-Dev/Documents/1PhD/Dataset_trials/B3_mimic/fasta/validation_summary.csv


### 2. Create a DF with the file names and their directory

In [ ]:
# Specify the folder path
folder_path = '/Users/MiladM-Dev/Documents/1PhD/Dataset_trials/B3_mimic/fasta'

# Get the list of filenames in the folder
file_names = os.listdir(folder_path)

# Create a DataFrame
df_files = pd.DataFrame(file_names, columns=['FileName'])

# Add a column with the full path for each file
df_files['FullPath'] = [os.path.abspath(os.path.join(folder_path, file)) for file in file_names]

# Display the DataFrame
df_files.to_csv('/Users/MiladM-Dev/Documents/1PhD/Dataset_trials/B3_mimic/fasta/filename_directory.csv')


### 3. Split the ID patient code from Sampling Abb.

1. Split the file names into 'prefix' and 'Remainder'
2. Remove '.fasta' 
3. Make the 'FullPath' column at the end

In [ ]:
# Split 'SCount' into two columns
df_files[['SCount_Prefix', 'SCount_Remainder']] = df_files['FileName'].str.extract(r'(\d{2}-\d{5})(.*)')
df_files['SCount_Remainder'] = df_files['SCount_Remainder'].str.replace('.fasta', '')

# Make the full path column always at the end
columns_order = [col for col in df_files.columns if col != 'FullPath'] + ['FullPath']
df_files = df_files[columns_order]
df_files.tail()

,FileName,SCount_Prefix,SCount_Remainder,FullPath
10,07-00235 copy.fasta,07-00235,copy,/Users/MiladM-Dev/Documents/1PhD/Dataset_trial...
11,06-00025.fasta,06-00025,,/Users/MiladM-Dev/Documents/1PhD/Dataset_trial...
12,07-00011R (FTA).fasta,07-00011,R (FTA),/Users/MiladM-Dev/Documents/1PhD/Dataset_trial...
13,07-00450.02.fasta,07-00450,.02,/Users/MiladM-Dev/Documents/1PhD/Dataset_trial...
14,06-00058 Ra.fasta,06-00058,Ra,/Users/MiladM-Dev/Documents/1PhD/Dataset_trial...


### 4. Excel file
1. Read CSV (or convert excel file to csv)
2. Extract the needed information (Optional)
3. Splir name to yy.nnnn remove .xx
4.


#### 1. Read CSV (or Convert xslx to csv)

In [ ]:
B3_excel = '/Users/MiladM-Dev/Documents/1PhD/Dataset_trials/B3_mimic/B3_example.csv'
B3xsl_df = pd.read_csv(B3_excel, encoding='utf-8', sep=';')
B3xsl_df.head()

,SCount,Ein,EingangsDatum,EntnahmeDatum,ErkrBeginn,EkzBeginn,BLand,GesAmt,Material,Genotyp,WHO distinct SeqID
0,06-00025.01,"70794 Filderstadt, Gemeinschaftspraxis",2/3/2006,2/1/2006,1/28/2006,1/31/2006,Baden-Württemberg,NaN,oral fluid,B3,2254
1,06-00036.02,"70771 Leinenfelden-Echterdingen, Dr. med. Iris...",2/10/2006,2/8/2006,2/4/2006,2/8/2006,Baden-Württemberg,NaN,oral fluid,B3,2254
2,06-00048.02,"70794 Filderstadt-Bonlanden, Gemeinnütziges Ge...",2/14/2006,2/6/2006,1/29/2006,2/5/2006,Baden-Württemberg,NaN,Rachenabstrich,B3,2254
3,06-00050.01,"70794 Filderstadt, Gemeinschaftspraxis",2/15/2006,2/14/2006,2/9/2006,2/11/2006,Baden-Württemberg,NaN,Rachenabstrich,B3,2254
4,06-00054.01,"70190 Stuttgart, Dr. med. Christoph Michels",2/20/2006,2/17/2006,2/15/2006,NaN,Baden-Württemberg,NaN,Rachenabstrich in Flüssigkeit,B3,2254


#### 2. Extract needed information

In [ ]:
B3_material = B3xsl_df[['SCount', 'Material']]
B3_material.head()

,SCount,Material
0,06-00025.01,oral fluid
1,06-00036.02,oral fluid
2,06-00048.02,Rachenabstrich
3,06-00050.01,Rachenabstrich
4,06-00054.01,Rachenabstrich in Flüssigkeit


#### 3. Split 'SCount' into two columns

1. Split *yy-nnnn.xx* to *yy-nnnn* and suffix
2. Drop the suffix column (.xx)

In [ ]:
# Split 'SCount' into two columns
B3_material[['SCount_Prefix', 'SCount_Suffix']] = B3_material['SCount'].str.extract(r'(\d{2}-\d{5})(.*)')

# Drop the suffix (.xx)
del B3_material['SCount_Suffix']
B3_material.head()

/var/folders/c5/srv5gzkd52l_9y41mpknm0t80000gp/T/ipykernel_34754/1226027365.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  B3_material[['SCount_Prefix', 'SCount_Suffix']] = B3_material['SCount'].str.extract(r'(\d{2}-\d{5})(.*)')
/var/folders/c5/srv5gzkd52l_9y41mpknm0t80000gp/T/ipykernel_34754/1226027365.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  B3_material[['SCount_Prefix', 'SCount_Suffix']] = B3_material['SCount'].str.extract(r'(\d{2}-\d{5})(.*)')


,SCount,Material,SCount_Prefix
0,06-00025.01,oral fluid,06-00025
1,06-00036.02,oral fluid,06-00036
2,06-00048.02,Rachenabstrich,06-00048
3,06-00050.01,Rachenabstrich,06-00050
4,06-00054.01,Rachenabstrich in Flüssigkeit,06-00054


### 4. Merging the Dfs (excel and seq files)

In [ ]:
# Merge DataFrames on 'SCount' and handle duplicates
df_merged = pd.merge(df_files, B3_material, on='SCount_Prefix', how='outer', suffixes=('_df_files', '_B3_material'))
df_merged = df_merged[['SCount_Prefix', 'FileName', 'SCount', 
       'Material', 'SCount_Remainder']]
#print(df_merged.columns)
df_merged#.head()

,SCount_Prefix,FileName,SCount,Material,SCount_Remainder
0,06-00025,06-00025.fasta,06-00025.01,oral fluid,
1,06-00036,06-00036.fasta,06-00036.02,oral fluid,
2,06-00048,NaN,06-00048.02,Rachenabstrich,NaN
3,06-00050,NaN,06-00050.01,Rachenabstrich,NaN
4,06-00054,NaN,06-00054.01,Rachenabstrich in Flüssigkeit,NaN
5,06-00057,NaN,06-00057.01,oral fluid,NaN
6,06-00058,06-00058 OF.fasta,06-00058.01,oral fluid,OF
7,06-00058,06-00058 Ra.fasta,06-00058.01,oral fluid,Ra
8,07-00011,07-00011R (FTA).fasta,07-00011.01,cDNA,R (FTA)
9,07-00235,07-00235 copy.fasta,NaN,NaN,copy


# 3. Create a Dictionary

In order to standardize the naming of Material across the df. Then keep the IDs with similar sampling types by splitting the df into 2 other df (Present and Not present in folder)
1. Get the uniqe names in each column
2. Create a dictionary out of this uniqe material names
3. Classify the sample types 
4. Drop the other 2 columns with different naming
5. splitting the df into 2 dfs (Present and Not present in folder)

#### 1. Get the uniqe names in each column

In [ ]:
# Get unique values from 'Material' and 'SCount_Remainder'
unique_material = list(df_merged["Material"].dropna().unique())
unique_scount_remainder = list(df_merged["SCount_Remainder"].dropna().unique())

# Print without quotes
print(unique_material)
print(unique_scount_remainder)

['oral fluid', 'Rachenabstrich', 'Rachenabstrich in Flüssigkeit', 'cDNA']
['', ' OF', ' Ra', 'R (FTA)', ' copy', '.02', 'Urin', 'Rsp', 'Rsp copy', 'Rspcopy copy 2', 'Rspcopy copy']


#### 2. Create a dictionary for this uniqe Abb.

In [ ]:
material_dict = {
    "Rachenabstrich": ["Rachenabstrich", 'Rsp', "RA", "Ra",'Rachenabstrich 1', 'ZKÜ-Ra','FTA card Ra', 'Rachenabstrich in Gel','cDNA aus Rachenabstrich','Rachenabstrich trocken', 'Filterpapier RA', 'RNA aus Rachenabstrich','Rachenabstrich in Flüssigkeit'] ,

    "Urine": ["Urine", "Urin", "Urin 1", 'ZKÜ-Urin (kein Mat. in MaMuRoe)','ZKÜ-Urin', 'cDNA aus Urin', 'Urinsediment in DMEM', 'RNA Urin'],

    'Serum': ['Serum', 'RNA Serum', 'Bserum', 'Serum (S laut MaMuRoe)'],

    'Plasma': ['RNA aus Plasma', 'Plasma'],

    'Oral Fluid': ['OF', 'oral fluid', 'cDNA aus Oral fluid', 'RNA aus Oral fluid']
}
df_materil_dict = pd.DataFrame(list(material_dict.items()), columns=['Material', 'Material Variants'])

df_materil_dict
#df_materil_dict.to_csv('/Users/MiladM-Dev/Documents/1PhD/meetings/Board Meeting/df_materil_dict.csv', index=False)

,Material,Material Variants
0,Rachenabstrich,"[Rachenabstrich, Rsp, RA, Ra, Rachenabstrich 1..."
1,Urine,"[Urine, Urin, Urin 1, ZKÜ-Urin (kein Mat. in M..."
2,Serum,"[Serum, RNA Serum, Bserum, Serum (S laut MaMuR..."
3,Plasma,"[RNA aus Plasma, Plasma]"
4,Oral Fluid,"[OF, oral fluid, cDNA aus Oral fluid, RNA aus ..."


#### 3. Mapping dictionary with DF

Classify each Material and Unify the material names across the DF

In [ ]:
def map_to_key(description):
    if not isinstance(description, str):  # Check if it's not a string
        description = str(description) if pd.notna(description) else ""  # Convert to string or empty string if NaN
    for key, values in material_dict.items():
        if any(value in description for value in values):
            return key
    return 'Unknown'

df_merged['Material dict'] = df_merged["Material"].apply(map_to_key)
df_merged['SCount_Remainder dict'] = df_merged["SCount_Remainder"].apply(map_to_key)
df_merged

,SCount_Prefix,FileName,SCount,Material,SCount_Remainder,Material dict,SCount_Remainder dict
0,06-00025,06-00025.fasta,06-00025.01,oral fluid,,Oral Fluid,Unknown
1,06-00036,06-00036.fasta,06-00036.02,oral fluid,,Oral Fluid,Unknown
2,06-00048,NaN,06-00048.02,Rachenabstrich,NaN,Rachenabstrich,Unknown
3,06-00050,NaN,06-00050.01,Rachenabstrich,NaN,Rachenabstrich,Unknown
4,06-00054,NaN,06-00054.01,Rachenabstrich in Flüssigkeit,NaN,Rachenabstrich,Unknown
5,06-00057,NaN,06-00057.01,oral fluid,NaN,Oral Fluid,Unknown
6,06-00058,06-00058 OF.fasta,06-00058.01,oral fluid,OF,Oral Fluid,Oral Fluid
7,06-00058,06-00058 Ra.fasta,06-00058.01,oral fluid,Ra,Oral Fluid,Rachenabstrich
8,07-00011,07-00011R (FTA).fasta,07-00011.01,cDNA,R (FTA),Unknown,Unknown
9,07-00235,07-00235 copy.fasta,NaN,NaN,copy,Unknown,Unknown


Drop columns of incosistant naming 

In [ ]:
#del df_merged["Material"]
#del df_merged["SCount_Remainder"]
df_merged.head()

,SCount_Prefix,FileName,SCount,Material dict,SCount_Remainder dict
0,06-00025,06-00025.fasta,06-00025.01,Oral Fluid,Unknown
1,06-00036,06-00036.fasta,06-00036.02,Oral Fluid,Unknown
2,06-00048,NaN,06-00048.02,Rachenabstrich,Unknown
3,06-00050,NaN,06-00050.01,Rachenabstrich,Unknown
4,06-00054,NaN,06-00054.01,Rachenabstrich,Unknown


In [ ]:
# Create two separate dataframes based on matching 'Material dict' and 'SCount_Remainder dict'
df_matched = df_merged[df_merged['Material dict'] == df_merged['SCount_Remainder dict']]
df_unmatched = df_merged[df_merged['Material dict'] != df_merged['SCount_Remainder dict']]

df_matched.head()
df_unmatched.head()

,SCount_Prefix,FileName,SCount,Material dict,SCount_Remainder dict
0,06-00025,06-00025.fasta,06-00025.01,Oral Fluid,Unknown
1,06-00036,06-00036.fasta,06-00036.02,Oral Fluid,Unknown
2,06-00048,NaN,06-00048.02,Rachenabstrich,Unknown
3,06-00050,NaN,06-00050.01,Rachenabstrich,Unknown
4,06-00054,NaN,06-00054.01,Rachenabstrich,Unknown
